In [ ]:
#Import all the libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
#Import the data
df = pd.read_csv('exoplanet_data.csv')

In [ ]:
df['koi_disposition'].unique()

In [ ]:
mapping = {'CONFIRMED':1,'FALSE POSITIVE':2,'CANDIDATE':3}
df.replace({'koi_disposition':mapping},inplace=True)
df.head()

In [ ]:
#This feature set will not have the error features.
dataset = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
                     'koi_fpflag_ec', 'koi_period', 'koi_impact',
                     'koi_duration', 'koi_depth', 'koi_prad',
                     'koi_teq', 'koi_model_snr', 'koi_steff',
                     'koi_slogg', 'koi_srad',
                     'ra', 'dec', 'koi_kepmag',
                     'koi_disposition']]

In [ ]:
#Create X and y
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1].values

In [ ]:
mms = MinMaxScaler()
X_mms = mms.fit_transform(X)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
gsc = GridSearchCV(
        estimator=SVR(kernel='rbf'),
        param_grid={
            'C': [0.1, 1, 100],
            'epsilon': [0.0001, 0.001, 0.01, 0.1, 1, 10],
            'gamma': [0.0001, 0.1, 10]
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

In [ ]:
grid_result = gsc.fit(X_mms, y)
best_params = grid_result.best_params_
best_svr = SVR(kernel='rbf', C=best_params["C"], epsilon=best_params["epsilon"], gamma=best_params["gamma"],
                   coef0=0.1, shrinking=True,
                   tol=0.001, cache_size=200, verbose=False, max_iter=-1)